In [1]:
import urllib.request, urllib.error, urllib.parse
import csv
from bs4 import BeautifulSoup
import html2text
import os
from datetime import datetime
import numpy as np
import pandas as pd
import json
from urllib.request import urlopen
from PIL import Image

In [125]:
def isnan(x):
    return x!=x

In [266]:
# function to open url and return it in squared format 

def getsquared_image(url):
    # open image-url, get width, height
    img = Image.open(urlopen(url))
    width, height = img.size
    # store the bigger one in s
    if width < height:
        s = height
    else:
        s = width
    
    # create empty opaque image file and paste picture
    new_img = Image.new('RGBA', (s,s), (255, 0, 0, 0))
    new_img.paste(img, (round((s-width)/2), round((s-height)/2)))
    
    return new_img

In [26]:
img = getsquared_image('https://images.metmuseum.org/CRDImages/md/web-large/DP169508.jpg')
#img

## load artist_qid

In [115]:
# open artist-metadata csv
filename = '../data/artist_qid_html_new.csv'
#csv_file = csv.reader(open(filename, "r"))
#df_new = pd.read_csv(filename)
df_new.to_csv(filename, index = False, header = True)
df_new.columns

Index(['Unnamed: 0', 'Name', 'Wikidata-Q-ID', 'Wikipedia-URL', 'URL-names',
       'Summary', 'Wikipedia-json', 'image', 'thumbnail', 'birth', 'death',
       'timeline_position', 'VIAF', 'wikidata', 'MET Object Indices',
       'REAL MET Object IDS', 'cloudinary thumbnails', 'URLs MET',
       'cloudinary artworks', 'Rijks name', 'URLs Rijk', 'Rijks img dates',
       'Rijks img PIDs', 'Rijks img Inrs', 'extract_html', 'URLs Rijk new',
       'URLs MET new'],
      dtype='object')

In [75]:
filename = '../data/artist_qid_rijks.csv'
#csv_file = csv.reader(open(filename, "r"))
#df_urls = pd.read_csv(filename)
#df['URLs Rijk'] = df_urls['URLs Rijk']
#df['URLs Rijk'][288].strip('[').strip(']').strip('\'')

In [235]:
df_new['cloudinary artworks'][4]

"['https://res.cloudinary.com/dfyzsulq8/image/upload/v1621499134/artworks/4_1_lnqpbc.png', 'https://res.cloudinary.com/dfyzsulq8/image/upload/v1621499134/artworks/4_3_ep59yl.png', 'https://res.cloudinary.com/dfyzsulq8/image/upload/v1621499134/artworks/4_2_sx4sol.png', 'https://res.cloudinary.com/dfyzsulq8/image/upload/v1621499134/artworks/4_0_s3iitv.png']"

In [237]:
# clean the formats in Rijks URLs list

ix_clean = []
for i in df_new['cloudinary artworks']:
    rix = []
    if i == '[]':
        rix = []
        ix_clean.append(rix)
    elif i!=i:
        rix = []
        ix_clean.append(rix)
    elif len(i.split(',')) == 1:
        rix = [(str(i).strip('[').strip(']').strip("'"))]
        ix_clean.append(rix)
        #print(int(str(i).strip('[').strip(']').strip("'")))
    elif len(i.split(',')) >1:
        for j in i.split(','):
            rix.append(j.strip('[').strip(']').strip("'").strip("\"").strip(" \'"))
        
        ix_clean.append(rix)
        
len(ix_clean)    
#rijks

292

In [239]:
df_new['cloudinary artworks new'] = ix_clean
df_new['cloudinary artworks new'][4]

['https://res.cloudinary.com/dfyzsulq8/image/upload/v1621499134/artworks/4_1_lnqpbc.png',
 'https://res.cloudinary.com/dfyzsulq8/image/upload/v1621499134/artworks/4_3_ep59yl.png',
 'https://res.cloudinary.com/dfyzsulq8/image/upload/v1621499134/artworks/4_2_sx4sol.png',
 'https://res.cloudinary.com/dfyzsulq8/image/upload/v1621499134/artworks/4_0_s3iitv.png']

# still formatting

In [124]:
# place two artworks into thumbnails
for i, j in enumerate(df_new['thumbnail']):
    if j!=j:
        #print(df_new['Name'][i],i)
        if df_new['URLs Rijk new'][i] != []:
            #df_new['thumbnail'][i] = df_new['URLs Rijk new'][i][0]
            print (df_new['thumbnail'][i])
         #   print(df['Name'][i])
    


# Download thumbnails

In [212]:
index = 284
index

284

In [213]:
# open thumbnail, get width, height

# limit the iterator (8 per call)

for i in range(8):
    # load file
    if isnan(df_new['thumbnail'][(int(index))]): # sort nan values out
        index += 1
        if index < 10:
            filename = '../data/thumbnails/thumbn_c/thumbnailDUMMY_00' + str(index) + '.png'
        elif index < 100:
            filename = '../data/thumbnails/thumbn_c/thumbnailDUMMY_0' + str(index) + '.png'
        else:
            filename = '../data/thumbnails/thumbn_c/thumbnailDUMMY_' + str(index) + '.png'
        
    else:
        new_img = getsquared_image(df_new['thumbnail'][((index))])
    
    # filename
        if index < 10:
            filename = '../data/thumbnails/thumbn_c/thumbnail_00' + str(index) + '.png'
        elif index < 100:
            filename = '../data/thumbnails/thumbn_c/thumbnail_0' + str(index) + '.png'
        else:
            filename = '../data/thumbnails/thumbn_c/thumbnail_' + str(index) + '.png'
        index +=1
    new_img.save(filename, 'PNG')   

In [219]:
list_of_cloud_thumbnail_filenames = os.listdir('../data/thumbnails2_cloudinary/')

In [224]:
# build filename list for .startswith comparison
index2 = 0
filename_list = []
for i in range(292):
    
    if index2 < 10:
        filename = "thumbnail_00" + str(index2) 
    elif index2 < 100:
        filename = "thumbnail_0" + str(index2)
    else:
        filename = "thumbnail_" + str(index2)
    index2+=1
    filename_list.append(filename)
#filename_list

In [222]:
#https://res.cloudinary.com/dfyzsulq8/image/upload/v1622639726/thumbnails2/thumbnail_291_fehaff.png
# get the fitting cloudinary filename for the thumbnails in a column
new_list = []
for i in filename_list:
    check = 0
    for j in list_of_cloud_thumbnail_filenames:
        if j.startswith(i):
            new_list.append('https://res.cloudinary.com/dfyzsulq8/image/upload/v1622639726/thumbnails2/'+j)
            check = 1
    if check == 0:
        new_list.append('')

#new_list
df_new['cloudinary thumbnails new'] = new_list


In [241]:
type(df_new['cloudinary artworks new'][4])

list

In [321]:
df_new.to_csv('BACKUP_df_NEW.csv', index=False, header=True)

In [243]:
df_new.columns

Index(['Unnamed: 0', 'Name', 'Wikidata-Q-ID', 'Wikipedia-URL', 'URL-names',
       'Summary', 'Wikipedia-json', 'image', 'thumbnail', 'birth', 'death',
       'timeline_position', 'VIAF', 'wikidata', 'MET Object Indices',
       'REAL MET Object IDS', 'cloudinary thumbnails', 'URLs MET',
       'cloudinary artworks', 'Rijks name', 'URLs Rijk', 'Rijks img dates',
       'Rijks img PIDs', 'Rijks img Inrs', 'extract_html', 'URLs Rijk new',
       'URLs MET new', 'cloudinary thumbnails new', 'cloudinary artworks new'],
      dtype='object')

# get all the Rijksmuseum artworks

In [265]:
df_new['URLs Rijk new'][30] == []

True

In [268]:
# open all the URLs and save the squared images names [artist-index]_[image-index].png

count = 0

# iterate over artists
for i in range(len(df_new)):
    
    if df_new['URLs Rijk new'][i] == []:
        print(count, i, df_new['URLs Rijk new'][i])
        count += 1

    else:
        
        index = 0
        # iterate over list of urls
        for j in df_new['URLs Rijk new'][i]:
    
            if j == 'nan':
                index += 1
                continue
            elif j == '':
                index += 1
                continue
            else:
                img = getsquared_image(j + '-s1400') #.replace(' ', '%20')
                filename = '../data/Rijks/' + str(i) + '_' + str(index) + '.png'
                img.save(filename, 'PNG')
                index += 1
        
        print(count)
        count += 1
 

0 0 []
1 1 []
2 2 []
3 3 []
4 4 []
5 5 []
6 6 []
7 7 []
8 8 []
9 9 []
10 10 []
11 11 []
12 12 []
13 13 []
14 14 []
15 15 []
16 16 []
17 17 []
18 18 []
19 19 []
20 20 []
21 21 []
22 22 []
23 23 []
24 24 []
25
26 26 []
27 27 []
28
29 29 []
30 30 []
31 31 []
32 32 []
33 33 []
34 34 []
35 35 []
36 36 []
37 37 []
38 38 []
39 39 []
40 40 []
41 41 []
42 42 []
43
44 44 []
45 45 []
46 46 []
47 47 []
48 48 []
49 49 []
50 50 []
51 51 []
52 52 []
53 53 []
54 54 []
55 55 []
56 56 []
57 57 []
58 58 []
59 59 []
60
61 61 []
62 62 []
63 63 []
64 64 []
65 65 []
66 66 []
67 67 []
68 68 []
69 69 []
70 70 []
71 71 []
72 72 []
73 73 []
74 74 []
75 75 []
76 76 []
77 77 []
78
79 79 []
80 80 []
81 81 []
82 82 []
83 83 []
84 84 []
85 85 []
86 86 []
87 87 []
88 88 []
89 89 []
90
91 91 []
92
93 93 []
94 94 []
95 95 []
96 96 []
97 97 []
98 98 []
99 99 []
100 100 []
101 101 []
102 102 []
103 103 []
104
105 105 []
106 106 []
107 107 []
108
109 109 []
110 110 []
111
112 112 []
113
114 114 []
115 115 []
116
117
118 11

In [263]:
for h in (df_new['URLs Rijk new'][25]):
    print(h)

https://lh3.googleusercontent.com/vIJFB18296VM5_ypebe56ALlvpSaOnRfatEiV_dUB12zzFs3t2Ta1bCoD-qK5eYON8fwfDLW3vb6R4MXzIxtAwuCCgAd=s0


In [274]:
df_new['Unnamed: 0'][0]

0

In [316]:
list_of_artworks = os.listdir('../data/Rijks_cloudinary/')
#list_of_artworks

['139_0_jykspw.png',
 '269_4_wbrlaw.png',
 '199_1_dcvftf.png',
 '291_1_wzvsqc.png',
 '116_30_qceoht.png',
 '252_2_m1irxk.png',
 '104_2_rfd1mc.png',
 '139_1_w0q0nl.png',
 '116_55_vl8nvg.png',
 '269_10_mvq4w7.png',
 '250_8_fcs0y6.png',
 '250_1_h5vxmk.png',
 '199_8_botxgq.png',
 '233_2_ix8z17.png',
 '116_15_p7ikh7.png',
 '25_0_uiwqih.png',
 '43_7_qpozpn.png',
 '78_0_tquwld.png',
 '238_7_db3oyi.png',
 '274_1_c1hrp7.png',
 '178_3_kxcirc.png',
 '116_0_iqmzq6.png',
 '199_6_vvpcx5.png',
 '255_2_tboihv.png',
 '238_4_wotvzn.png',
 '233_4_cngvrx.png',
 '116_9_yia6zu.png',
 '.DS_Store',
 '116_50_obgrmi.png',
 '116_2_vpbhaj.png',
 '238_9_u3fhq9.png',
 '199_4_eixvyy.png',
 '116_18_cgg2br.png',
 '280_3_aag6qa.png',
 '116_32_c0yoqb.png',
 '199_9_yb9wrz.png',
 '264_1_m2k2ek.png',
 '153_0_ywdm8k.png',
 '199_0_aeovl8.png',
 '250_4_vhum2n.png',
 '60_0_dqm1jj.png',
 '269_9_ntofwl.png',
 '43_5_wi8xs7.png',
 '157_0_n2tbqg.png',
 '116_22_myxtdw.png',
 '116_11_afwwos.png',
 '43_6_tplwxs.png',
 '291_0_hvazxk.pn

In [317]:
# get the fitting cloudinary filename for the Rijks artworks
# https://res.cloudinary.com/dfyzsulq8/image/upload/v1622644688/artworks_rijk/286_1_vdkwpx.png
artwork_list = []
for i in df_new['Unnamed: 0']:
    check = 0
    storage = []
    for j in list_of_artworks:
        if j.startswith(str(i) + '_'):
            storage.append('https://res.cloudinary.com/dfyzsulq8/image/upload/v1622644688/artworks_rijk/' + str(j))
            check = 1
    if check == 0:
        artwork_list.append('')
    else:
        artwork_list.append(storage)
df_new[' Rijks cloudinary artworks new'] = artwork_list
(artwork_list)

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 ['https://res.cloudinary.com/dfyzsulq8/image/upload/v1622644688/artworks_rijk/25_0_uiwqih.png'],
 '',
 '',
 ['https://res.cloudinary.com/dfyzsulq8/image/upload/v1622644688/artworks_rijk/28_0_mycyy8.png'],
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 ['https://res.cloudinary.com/dfyzsulq8/image/upload/v1622644688/artworks_rijk/43_7_qpozpn.png',
  'https://res.cloudinary.com/dfyzsulq8/image/upload/v1622644688/artworks_rijk/43_5_wi8xs7.png',
  'https://res.cloudinary.com/dfyzsulq8/image/upload/v1622644688/artworks_rijk/43_6_tplwxs.png',
  'https://res.cloudinary.com/dfyzsulq8/image/upload/v1622644688/artworks_rijk/43_3_bclj0r.png',
  'https://res.cloudinary.com/dfyzsulq8/image/upload/v1622644688/artworks_rijk/43_0_erxfo7.png',
  'https://res.cloudinary.com/dfyzsulq8/image/upload/v1622644688/artworks_rijk/43_1_j9dssv.png',
  'https://res.cl

In [314]:
df_new[' Rijks cloudinary artworks new']

0       
1       
2       
3       
4       
      ..
287     
288     
289     
290     
291     
Name:  Rijks cloudinary artworks new, Length: 292, dtype: object

# finally make the artwork column!

In [283]:
df_new.columns

Index(['Unnamed: 0', 'Name', 'Wikidata-Q-ID', 'Wikipedia-URL', 'URL-names',
       'Summary', 'Wikipedia-json', 'image', 'thumbnail', 'birth', 'death',
       'timeline_position', 'VIAF', 'wikidata', 'MET Object Indices',
       'REAL MET Object IDS', 'cloudinary thumbnails', 'URLs MET',
       'cloudinary artworks', 'Rijks name', 'URLs Rijk', 'Rijks img dates',
       'Rijks img PIDs', 'Rijks img Inrs', 'extract_html', 'URLs Rijk new',
       'URLs MET new', 'cloudinary thumbnails new', 'cloudinary artworks new',
       ' Rijks cloudinary artworks new'],
      dtype='object')

In [318]:
final_storage = []

for i, j in enumerate(df_new['cloudinary thumbnails new']):
    store = list()
    store.append(j)
    for k in df_new['cloudinary artworks new'][i]:
        store.append(k)
    for l in df_new[' Rijks cloudinary artworks new'][i]:
        store.append(l)
    #.extend().extend(df_new[' Rijks cloudinary artworks new'][i])
    final_storage.append(store)
len(final_storage)

292

In [358]:
df_new['cloudinary thumbnails new']

0      https://res.cloudinary.com/dfyzsulq8/image/upl...
1      https://res.cloudinary.com/dfyzsulq8/image/upl...
2      https://res.cloudinary.com/dfyzsulq8/image/upl...
3      https://res.cloudinary.com/dfyzsulq8/image/upl...
4      https://res.cloudinary.com/dfyzsulq8/image/upl...
                             ...                        
287    https://res.cloudinary.com/dfyzsulq8/image/upl...
288    https://res.cloudinary.com/dfyzsulq8/image/upl...
289    https://res.cloudinary.com/dfyzsulq8/image/upl...
290    https://res.cloudinary.com/dfyzsulq8/image/upl...
291    https://res.cloudinary.com/dfyzsulq8/image/upl...
Name: cloudinary thumbnails new, Length: 292, dtype: object

In [ ]:
thumbnail = ''
for i in df['']:
    death+=str(i)+','
    
death = death.strip(',')

In [359]:
#final_storage[233]
with open('')

In [320]:
df_new['final artworks'] = final_storage


# END of formatting make final cables dataframe

In [323]:
#final_storage

In [327]:
position_in_timeline = [1271,1266,1650,1254,1282,1249,1276,1268,1304,1400,1316,1311,1326,1305,1314,1319,1308,1314.5,1285,1309,1324,1328,1341,1340,1412,1373,1395,1365,1287,1395.5,1321,1380,1384,1319.5,1398,1392,1388,1407,1453,1406,1410,1446,1396.0,1440,1480,1527,1436,1416,1405,1415,1420,1414,1411,1426,1434,1439,1429,1448,1461,1454,1424,1449,1487,1438,1429.5,1449.5,1462,1466,1433,1465,1468,1458,1655,1463,1472,1433.5,1431,1390,1425,1376,1435,1453.5,1461.5,1470,1441,1443,1457,1448.5,1431.5,1447,1456,1498,1504,1474,1430.0,1473,1463.5,1475,1514,1468.5,1467,1470.5,1477,1471,1466.5,1478,1488,1486,1464,1469,1445,1473.5,1465.5,1457.5,1490,1535,1464.5,1475.5,1482,1462.5,1499,1484,1485,1481,1479,1526,1510,1500,1496,1407.5,1383,1360,1404,1335,1355,1467.5,1495,1483,1495.5,1469.5,1491,1492,1481.5,1487.5,1496.5,1494,1507,1521,1521.5,1483.5,1518,1522,1501,1478.5,1532,1482.5,1489,1512,1488.5,1474.5,1489.5,1490.5,1493,1497.0,1476.0,1491.5,1502,1497.5,1477.5,1484.5,1476.5,1505,1492.5,1511,1493.5,1494.5,1509,1498.5,1530,1499.5,1500.5,1516,1508,1501.5,1503,1503.5,1505.5,1556,1567,1580,1517,1530.5,1520,1506,1519,1507.5,1512.5,1513,1515,1516.5,1504.5,1508.5,1509.5,1502.5,1506.5,1517.5,1510.5,1513.5,1471.5,1511.5,1514.5,1515.5,1524,1518.5,1519.5,1520.5,1522.5,1523.0,1523.5,1524.5,1525.0,1525.5,1526.5,1542,1527.5,1528.0,1528.5,1529.0,1529.5,1531.0,1538,1531.5,1532.5,1558,1565,1533.0,1533.5,1534.0,1534.5,1535.5,1536,1536.5,1485.5,1479.5,1537.0,1537.5,1551,1540,1538.5,1539.0,1539.5,1548,1540.5,1543,1574,1541.0,1543.5,1544,1541.5,1561,1554,1542.5,1544.5,1550,1562,1545.0,1545.5,1557,1553,1560,1546.0,1546.5,1557.5,1547.0,1547.5,1548.5,1549.0,1566,1594,1549.5,1550.5,1551.5,1552.0,1552.5,1660,1553.5,1554.5,1555.0,1575,1555.5,1556.5,1558.5]
len(position_in_timeline)

292

In [328]:
# make new dataframe for cables -> to be converted to json array

index = []
for i in range(292):
    index.append(i)
df_cables = pd.DataFrame()
df_cables['index'] = index
df_cables['Name'] = df_new['Name']
df_cables['birth'] = df_new['birth']
df_cables['death'] = df_new['death']
df_cables['timeline_position'] = df_new['timeline_position']
df_cables['timeline_position_extended'] = position_in_timeline
df_cables['Wikipedia'] = df_new['Wikipedia-URL']
df_cables['Summary'] = df_new['Summary']
df_cables['extract_html'] = df_new['extract_html']                     # make new!
df_cables['cloudinary thumbnails'] = df_new['cloudinary thumbnails new'] # make new!
df_cables['final artworks'] = final_storage   # make new!


df_cables['Summary'][2]

'Bonanno Pisano (Pisa), active in the 1170s and 1180s, was an Italian sculptor, mixing Byzantine and classical elements. Giorgio Vasari attributed the realization of the Leaning Tower of Pisa to him in his Vite. Pisano was born in Pisa and worked there most of his life. In the 1180s, he departed for Monreale, in Sicily, where he completed the doors to the cathedral before returning to Pisa, where he died. Pisano was buried at the foot of the leaning tower, where his sarcophagus was discovered in 1820.'

In [330]:
df_final = df_cables.sort_values(by = ['timeline_position_extended'], inplace= False)
df_final

,index,Name,birth,death,timeline_position,timeline_position_extended,Wikipedia,Summary,extract_html,cloudinary thumbnails,final artworks
5,5,Andrea Tafi,1213,1294,1254,1249.0,https://en.wikipedia.org/wiki/Andrea_Tafi_(art...,Andrea Tafi (1213–1294) was an Italian artist....,<p><b>Andrea Tafi</b> (1213–1294) was an Itali...,https://res.cloudinary.com/dfyzsulq8/image/upl...,[https://res.cloudinary.com/dfyzsulq8/image/up...
3,3,Nicola Pisano,1225,1284,1254,1254.0,https://en.wikipedia.org/wiki/Nicola_Pisano,Nicola Pisano was an Italian sculptor whose wo...,<p><b>Nicola Pisano</b> was an Italian sculpto...,https://res.cloudinary.com/dfyzsulq8/image/upl...,[https://res.cloudinary.com/dfyzsulq8/image/up...
1,1,Arnolfo di Lapo,1240,1302,1271,1266.0,https://en.wikipedia.org/wiki/Arnolfo_di_Cambio,Arnolfo di Cambio was an Italian architect and...,<p><b>Arnolfo di Cambio</b> was an Italian arc...,https://res.cloudinary.com/dfyzsulq8/image/upl...,[https://res.cloudinary.com/dfyzsulq8/image/up...
7,7,Margaritone,1245,1290,1268,1268.0,https://en.wikipedia.org/wiki/Margaritone_d%27...,"Margarito, Margaritone da Arezzo, or Margarito...","<p><b>Margarito</b>, Margaritone da Arezzo, or...",https://res.cloudinary.com/dfyzsulq8/image/upl...,[https://res.cloudinary.com/dfyzsulq8/image/up...
0,0,Cimabue,1240,1302,1271,1271.0,https://en.wikipedia.org/wiki/Cimabue,"Cimabue, also known as Cenni di Pepo or Cenni ...","<p><b>Cimabue</b>, also known as <b>Cenni di P...",https://res.cloudinary.com/dfyzsulq8/image/upl...,[https://res.cloudinary.com/dfyzsulq8/image/up...
...,...,...,...,...,...,...,...,...,...,...,...
189,189,Sofonisba Anguissola,1534,1625,1580,1580.0,https://en.wikipedia.org/wiki/Sofonisba_Anguis...,"Sofonisba Anguissola, also known as Sophonisba...","<p><b>Sofonisba Anguissola</b>, also known as ...",https://res.cloudinary.com/dfyzsulq8/image/upl...,[https://res.cloudinary.com/dfyzsulq8/image/up...
278,278,Bartolomeo Ammannati,1596,1592,1594,1594.0,https://en.wikipedia.org/wiki/Bartolomeo_Amman...,Bartolomeo Ammannati was an Italian architect ...,<p><b>Bartolomeo Ammannati</b> was an Italian ...,https://res.cloudinary.com/dfyzsulq8/image/upl...,[https://res.cloudinary.com/dfyzsulq8/image/up...
2,2,Bonanno,9999,9999,9999,1650.0,https://en.wikipedia.org/wiki/Bonanno_Pisano,"Bonanno Pisano (Pisa), active in the 1170s and...","<p><b>Bonanno Pisano</b> (Pisa), active in the...",https://res.cloudinary.com/dfyzsulq8/image/upl...,[https://res.cloudinary.com/dfyzsulq8/image/up...
72,72,Mino del Reame,9999,9999,9999,1655.0,https://en.wikipedia.org/wiki/Mino_del_Reame,"Mino del Reame, also known as Mino dal Reame, ...","<p><b>Mino del Reame</b>, also known as <b>Min...",https://res.cloudinary.com/dfyzsulq8/image/upl...,[https://res.cloudinary.com/dfyzsulq8/image/up...


In [339]:
df_final.to_csv('FINAL.csv', index = False, header = True)

In [332]:
len(df_final)

292

In [342]:
df_final.to_json('FINAL_json_indexf.json', orient = 'index')

In [347]:
os.listdir('../')

['a_test_291.png',
 'LICENSE',
 'diverse_jupyter_notebooks',
 'README.md',
 '.gitignore',
 'lives2.html',
 '.git',
 'data']

In [348]:
# make list of jsons from dataframe
list_of_json = []
for i in range(len(df_final2)):
    x = df_final2.iloc[i].to_json()
    list_of_json.append(x)
len(list_of_json)
#df_cables.iloc[4].to_json()

292

In [354]:
with open('json_listTEST.json', 'w+') as output:
    output.write(str(list_of_json))
    output.close()

In [357]:
another_list = []
for i in list_of_json:
    data = json.loads(i)
    another_list.append(data)
type(another_list[0])
with open('json_list_NEW_testtest.json', 'w') as outfile:
    json.dump(another_list, outfile)

In [350]:
(df_final2)['indexf'] = range(292)

In [351]:
df_final2 = df_final.set_index('indexf')
df_final2

,index,Name,birth,death,timeline_position,timeline_position_extended,Wikipedia,Summary,extract_html,cloudinary thumbnails,final artworks
indexf,,,,,,,,,,,
0,5,Andrea Tafi,1213,1294,1254,1249.0,https://en.wikipedia.org/wiki/Andrea_Tafi_(art...,Andrea Tafi (1213–1294) was an Italian artist....,<p><b>Andrea Tafi</b> (1213–1294) was an Itali...,https://res.cloudinary.com/dfyzsulq8/image/upl...,[https://res.cloudinary.com/dfyzsulq8/image/up...
1,3,Nicola Pisano,1225,1284,1254,1254.0,https://en.wikipedia.org/wiki/Nicola_Pisano,Nicola Pisano was an Italian sculptor whose wo...,<p><b>Nicola Pisano</b> was an Italian sculpto...,https://res.cloudinary.com/dfyzsulq8/image/upl...,[https://res.cloudinary.com/dfyzsulq8/image/up...
2,1,Arnolfo di Lapo,1240,1302,1271,1266.0,https://en.wikipedia.org/wiki/Arnolfo_di_Cambio,Arnolfo di Cambio was an Italian architect and...,<p><b>Arnolfo di Cambio</b> was an Italian arc...,https://res.cloudinary.com/dfyzsulq8/image/upl...,[https://res.cloudinary.com/dfyzsulq8/image/up...
3,7,Margaritone,1245,1290,1268,1268.0,https://en.wikipedia.org/wiki/Margaritone_d%27...,"Margarito, Margaritone da Arezzo, or Margarito...","<p><b>Margarito</b>, Margaritone da Arezzo, or...",https://res.cloudinary.com/dfyzsulq8/image/upl...,[https://res.cloudinary.com/dfyzsulq8/image/up...
4,0,Cimabue,1240,1302,1271,1271.0,https://en.wikipedia.org/wiki/Cimabue,"Cimabue, also known as Cenni di Pepo or Cenni ...","<p><b>Cimabue</b>, also known as <b>Cenni di P...",https://res.cloudinary.com/dfyzsulq8/image/upl...,[https://res.cloudinary.com/dfyzsulq8/image/up...
...,...,...,...,...,...,...,...,...,...,...,...
287,189,Sofonisba Anguissola,1534,1625,1580,1580.0,https://en.wikipedia.org/wiki/Sofonisba_Anguis...,"Sofonisba Anguissola, also known as Sophonisba...","<p><b>Sofonisba Anguissola</b>, also known as ...",https://res.cloudinary.com/dfyzsulq8/image/upl...,[https://res.cloudinary.com/dfyzsulq8/image/up...
288,278,Bartolomeo Ammannati,1596,1592,1594,1594.0,https://en.wikipedia.org/wiki/Bartolomeo_Amman...,Bartolomeo Ammannati was an Italian architect ...,<p><b>Bartolomeo Ammannati</b> was an Italian ...,https://res.cloudinary.com/dfyzsulq8/image/upl...,[https://res.cloudinary.com/dfyzsulq8/image/up...
289,2,Bonanno,9999,9999,9999,1650.0,https://en.wikipedia.org/wiki/Bonanno_Pisano,"Bonanno Pisano (Pisa), active in the 1170s and...","<p><b>Bonanno Pisano</b> (Pisa), active in the...",https://res.cloudinary.com/dfyzsulq8/image/upl...,[https://res.cloudinary.com/dfyzsulq8/image/up...


In [361]:
thumbnail = ''
for i in df_final2['cloudinary thumbnails']:
    thumbnail+=str(i)+','
    
death = thumbnail.strip(',')

In [363]:
f = open('thumbnails_squared.txt', 'wb') # names
f.write(death.encode("utf-8"))
f.close()

In [360]:
df_final2.columns

Index(['index', 'Name', 'birth', 'death', 'timeline_position',
       'timeline_position_extended', 'Wikipedia', 'Summary', 'extract_html',
       'cloudinary thumbnails', 'final artworks'],
      dtype='object')